# Financial project 

The objective of this notebook is to:

- What is the relation between investor sentiment on social media and stock market returns? Any predictive power?

Slide availables here: [Project](https://drive.google.com/open?id=0B0rdK44Elj9RZzdYMnJzaTREaHlhOUlmNGd2Qzg3RFJTSDBn)

The online codes are available [here](https://repl.it/@trenault/StockTwits101)

The API returns 30 messages at a time.

To get older messages need the specify the `max` arguments: https://api.stocktwits.com/developers/docs/api#streams-symbol-docs

- https://api.stocktwits.com/api/2/streams/symbol/max=177617138/BTC.X.json

Example

-  base URL: https://api.stocktwits.com/api/2/streams/symbol/

Paramters:

-  1) ticker
- 2) max id

- Only latest 30 messages
https://api.stocktwits.com/api/2/streams/symbol/BTC.X.json




We need the following information in a dataframe:

- `messages`
  - `id`
  - `body`
  - `created_at`
  - `user`
    - `id`
    - `username`
    - `name`
    - `avatar_url`
    - `join_date` 

## Stocktwit API

[API](https://api.stocktwits.com/developers/docs)

For next week, scrap  the data for bitcoin and messages about bitcoins

## MongoDB

Installation process: https://docs.mongodb.com/manual/tutorial/install-mongodb-on-os-x/

To open the database, please first open the terminal and paste the following code:

```
mongod --config /usr/local/etc/mongod.conf
```

Then open another terminal and launch mongoDB with `mongo` to use MongoDB with the command line

You are all set!

## Sentiment

If sentiment is Bullish, label +1, if sentiment is Bearish, lable -1 else 0

## Symbols

Extract the tickers in a list

In [1]:
import urllib3
import json
import pandas as pd
import datetime
import pymongo
from pymongo import MongoClient

import warnings
warnings.filterwarnings('ignore')

In [2]:
client = MongoClient('localhost', 27017)

### Client is the database
db = client['SorbonneBigData']

### Collection name
collection = db.stocktwits

### Add element to the collection
insert_element = {'id': "Hello", "ok": "Yes"}
result = collection.insert_one(insert_element)

In [3]:
def get_json_ticker(
        nameDatabase,
        collectionName,
        ticker, max_twit=False,
        to_mongodb=False):
    """
    ### Note that 
    - nameDatabase is the clienty
    - collectionName is the client and collection
    
    fonction to scrap message in stocktwit

    ## Step 1:
    - Extract single ticker

    ## Step 2:
    - Format to list

    ## Step 3:
    - Save to MongoDB

    """

    http = urllib3.PoolManager()

    if max_twit:
        url = "https://api.stocktwits.com/api/2/streams/symbol/{0}.json?max={1}".format(
            ticker,
            max_twit)
    else:
        url = "https://api.stocktwits.com/api/2/streams/symbol/{0}.json".format(
            ticker)

     # get the data
    r = http.request('GET', url)
    data = json.loads(r.data)

    if data["response"]["status"] == 200:
        #print("My request was accepted")

        #dic_scrap = {
        #    "id": [],
        #    "body": [],
        #    "sentiment": [],
        #    "userName": [],
        #    "symbols": []
        #}

        # Retrieve last ID
        
        
        for i, element in enumerate(data["messages"]):

            # Store data for the fun

            #id_message = dic_scrap['id'].append(element["id"])
            #body_message = dic_scrap['body'].append(element["body"])
            #sentiment = dic_scrap['sentiment'].append(element["entities"]["sentiment"]["basic"])
            #user_name = dic_scrap['userName'].append(element["user"]["username"])

            id_message = element["id"]
            body_message = element["body"]
            #sentiment = dic_scrap['sentiment'].append(element["entities"]["sentiment"]["basic"])
            user_name = element["user"]["username"]
            created_at = element["created_at"]
            
            ### Store sentiment
            sentiment =  element['entities']['sentiment']
            
            if sentiment:
                to_store_sentiment = sentiment['basic']
            else:
                to_store_sentiment = 'Neutral'
                
            ### Symobls
            #len_sym = len(element['symbols'])
            list_symbols = []
            
            ### itertate over the dictionnary to store the symbol

            list_symbols = []
            for sym in element['symbols']:
                list_symbols.append(sym['title'])                                    
                        ### Get different symbols

            if to_mongodb:

                # Client is the database
                db = nameDatabase

                # Collection name
                collection = collectionName
                
                if to_store_sentiment == 'Bullish':
                    to_store_sentiment_ = 1
                elif to_store_sentiment == 'Bearish':
                    to_store_sentiment_ = -1
                else:
                    to_store_sentiment_ = 0
                    
                    

                insert_element = {"id": id_message,
                                  "created_at": element["created_at"],
                                  "user": user_name,
                                  "sentiment": to_store_sentiment,
                                  "sentiment_":to_store_sentiment_, 
                                  "body": body_message,
                                  "symbols": list_symbols}

                result = collection.insert_one(insert_element)
                
            if i == 29:
                latest_id = element["id"]

                dic_ = {
                    'id': latest_id,
                    'url': url
                }
                
        return dic_
                
    else:
        print(data["response"]["status"])
        status = 429
        
        return status

    


Test the function with only one Get/Request

In [4]:
# Client is the database
#db = client['ThisIsATest']

# Collection name
#collection = db.Tweet

#last, data = 

#data = get_json_ticker(
#    nameDatabase=db,
#    collectionName=collection,
#    ticker='BTC.X',
#    max_twit=False,
#    to_mongodb=False)
#data

## Call API in loop

In [5]:
from datetime import datetime, timedelta
import time
from tqdm import tqdm

In [ ]:
list_id = []
list_url = []

# Client is the database
db = client['StockTwitClass101']

# Collection name
collection = db.messages

# tickers
ticker = 'BTC.X'

# Get the first 30 messages and store the ID
output = get_json_ticker(
    nameDatabase=db,
    collectionName=collection,
    ticker=ticker,
    max_twit=False,
    to_mongodb=True)

# Loop to get the messages

timesecond = 0
lastid = output['id']

for h in range(0,8):
    time.sleep(timesecond)
    begin = datetime.now()
    for i in tqdm(range(0, 200)):

        output = get_json_ticker(
                nameDatabase=db,
                collectionName=collection,
                ticker=ticker,
                max_twit=lastid,
                to_mongodb=True)
        
        if output == 429:
            end = datetime.now()
            time_code = end - begin
    
            time_next_batch = begin + timedelta(hours=1)
            time_end_batch = begin + timedelta(seconds=time_code.seconds)
            timesecond = (time_next_batch - time_end_batch).seconds
            
        else:
            
            lastid = output['id']
    
    print('Next batch in {} minutes. It will happen at {}'.format(timesecond/60, 
                                           time_next_batch.strftime("%H:%M:%S"))
         )

    #name = '{0}.csv'.format(datetime.datetime.now())
    # with open(name, 'w') as f:
    #json.dump(data, f)

    # list_id.append(last['id'])
    # list_url.append(last['url'])


100%|██████████| 200/200 [06:52<00:00,  1.64s/it]

429
Next batch in 53.13333333333333 minutes. It will happen at 16:56:10



 38%|███▊      | 75/200 [02:45<04:51,  2.33s/it]

# Upload data from MongoDB

In [ ]:
cursor = db.messages.find({
    "sentiment":'Bullish',
    "user":"YoungMula"
                          })
for document in cursor:
    print(document['sentiment'])

In [ ]:
cursor = db.messages.find({"sentimen()"})
for document in cursor:
    if len(document['symbols']) == 2: 
        print(document['body'])

# MongoDB 

In [ ]:
insert_element = {"id": 1,
                                  "created_at": 2,
                                  "user": 3,
                                  "sentiment": 4,
                                  "sentiment_":5, 
                                  "body": 6,
                                  "symbols": 7}

In [ ]:
insert_element['id']

In [ ]:
db = client['StockTwitClass101']
cursor = db.messages.find({})
#for document in cursor:
#    print(document)

In [ ]:
import pandas as pd
df = pd.DataFrame(list(db.messages.find({})))
list(df)

In [ ]:
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

In [ ]:
pos_msg = []

neg_msg = []

sentiment_list = []